## algorithm design and anlysis-2025 spring  homework 4
**Deadline**：2025.5.14

**name**:


note：
---
1. 带有\*的题目，申请免上课的同学，必须完成，其他同学选作；
2. 请独立完成，如求助了他人或者大模型，请著明，并且不可省略算法分析部分；
4. 如若作答有雷同，全部取消成绩；
3. 需要书面作答的题目，可以通过引用图片的形式添加，但是注意上传项目时包含所引用的图片的源文件；
4. $log_n$ 默认表示$log_2{n}$;

## 问题 1 
**最小生成树（Minimum Spanning Tree）**

设  **G**  为一个带权重的连通无向图，且所有边的权重均不相等。令$e_i$ 为权重第 $i$ 小的边。最小生成树（MST）是否必须包含 $e_1$ ? 同理，是否必须包含 $e_2$ 和 $e_3$ ? 若必须包含，请给出证明；否则，请构造反例。需从基本原理论证，不能依赖割引理(cut lemma) 或 Prim/Kruskal算法的正确性。


answer:

![问题1](./fig/hw4/问题1.png)

## 问题 2 
**瓶颈生成树（Bottleneck Spanning Tree）**

带有权重的无向图 $G(V,E,w)$ 的瓶颈生成树，表现为：在所有生成树中，最大权重边的权重值最小。即，BST $T$ 最小化瓶颈损失 $c(T)=max_{e \in T}{w(e)}$。

1. 证明 $G$ 的每一个最小生成树（MST）都是瓶颈生成树（BST）
2. 设计一个线性时间复杂度的算法：， 对于一个图 $G(V,E,w)$ 和一个整数 $b$，判断图 $ G$ 是否存在一个瓶颈生成树，其最大权重边的权重不超过 $b$，分析算法设计思路，并基于python编程实现。
3. 设计一个线性时间复杂度的算法：对于给定的图 $G(V,E,w)$，找到其瓶颈生成树，分析算法设计思路，并基于python编程实现。

idea：

![问题2](./fig/hw4/问题2.png)

### 算法设计思路
2. 判断是否存在瓶颈值不超过b的BST算法
- 将图中所有边权重小于或等于b的边组成一个子图G'
- 检查子图G'是否连通（是否包含所有原图中的顶点）
- 如果G'连通，则存在一个瓶颈生成树，其最大边权重不超过b
- 如果G'不连通，则不存在这样的瓶颈生成树

3. 找到瓶颈生成树算法
- 将图中所有边按权重从小到大排序
- 使用二分查找确定最小的权重b，使得由权重 不超过b的边构成的子图是连通的
- 在这个子图上运行BFS得到一颗瓶颈生成树

### 时间复杂度
2. 
- 筛选边：O(|E|)
- 检查连通性（通过BFS或DFS）：O(|V|+|E|)
- 总体时间复杂度：O(|V|+|E|)

3. 
- 使用桶排序预处理边：O(|E|)
- 从小到大按权重添加边，直到图连通：O(|V|+|E|)
- 总体时间复杂度：O(|V|+|E|)

In [1]:
# add your code here
# algorithm of the liear time complexity 

from collections import defaultdict, deque

class Graph:
    def __init__(self, vertices):
        self.V = vertices
        self.graph = defaultdict(list)
    
    def add_edge(self, u, v, w):
        self.graph[u].append((v, w))
        self.graph[v].append((u, w))
    
    def is_connected_with_threshold(self, threshold):
        if not self.graph:
            return False
        
        visited = [False] * self.V
        start_vertex = list(self.graph.keys())[0]
        
        queue = deque([start_vertex])
        visited[start_vertex] = True
        
        while queue:
            u = queue.popleft()
            for v, weight in self.graph[u]:
                if not visited[v] and weight <= threshold:
                    visited[v] = True
                    queue.append(v)
        
        return all(visited[i] for i in range(self.V) if i in self.graph)
    
    def has_bst_with_max_weight(self, b):
        return self.is_connected_with_threshold(b)
    
    def find_bottleneck_spanning_tree(self):
        if not self.graph: 
            return []
        
        all_weights = set()
        for u in self.graph:
            for v, w in self.graph[u]:
                if u < v: 
                    all_weights.add(w)
        
        weights_list = sorted(all_weights)
        
        left, right = 0, len(weights_list) - 1
        
        while left < right:
            mid = (left + right) // 2
            if self.is_connected_with_threshold(weights_list[mid]):
                right = mid
            else:
                left = mid + 1
        
        threshold = weights_list[left]
        
        bst_edges = []
        visited = [False] * self.V
        start_vertex = list(self.graph.keys())[0]
        
        queue = deque([(start_vertex, -1)])
        visited[start_vertex] = True
        
        while queue:
            u, parent = queue.popleft()
            
            for v, weight in self.graph[u]:
                if not visited[v] and weight <= threshold:
                    visited[v] = True
                    queue.append((v, u))
                    bst_edges.append((u, v, weight))
        
        return bst_edges

# 优化版本
def find_bottleneck_spanning_tree_linear(graph):
    edges = []
    max_weight = 0
    
    for u in graph.graph:
        for v, w in graph.graph[u]:
            if u < v:  
                edges.append((u, v, w))
                max_weight = max(max_weight, w)
    
    buckets = [[] for _ in range(max_weight + 1)]
    for u, v, w in edges:
        buckets[w].append((u, v))
    
    parent = {i: i for i in range(graph.V)}
    rank = {i: 0 for i in range(graph.V)}
    
    def find(x):
        if parent[x] != x:
            parent[x] = find(parent[x])
        return parent[x]
    
    def union(x, y):
        root_x = find(x)
        root_y = find(y)
        
        if root_x == root_y:
            return
        
        if rank[root_x] < rank[root_y]:
            parent[root_x] = root_y
        else:
            parent[root_y] = root_x
            if rank[root_x] == rank[root_y]:
                rank[root_x] += 1
    
    bst_edges = []
    components = graph.V
    bottleneck_weight = -1
    
    for weight in range(max_weight + 1):
        for u, v in buckets[weight]:
            if find(u) != find(v):
                union(u, v)
                bst_edges.append((u, v, weight))
                components -= 1
                bottleneck_weight = weight
        
        if components == 1:
            break
    
    return bst_edges, bottleneck_weight

def test():
    g = Graph(5)
    g.add_edge(0, 1, 2)
    g.add_edge(0, 3, 1)
    g.add_edge(1, 2, 3)
    g.add_edge(1, 3, 4)
    g.add_edge(1, 4, 5)
    g.add_edge(2, 4, 7)
    g.add_edge(3, 4, 6)
    
    print("瓶颈值不超过3的BST存在:", g.has_bst_with_max_weight(3))
    print("瓶颈值不超过2的BST存在:", g.has_bst_with_max_weight(2))
    print("瓶颈值不超过1的BST存在:", g.has_bst_with_max_weight(1))
    
    bst = g.find_bottleneck_spanning_tree()
    print("瓶颈生成树的边:", bst)
    
    # 线性时间算法
    bst_linear, bottleneck = find_bottleneck_spanning_tree_linear(g)
    print("线性时间算法找到的瓶颈生成树的边:", bst_linear)
    print("瓶颈值:", bottleneck)

if __name__ == "__main__":
    test()

瓶颈值不超过3的BST存在: False
瓶颈值不超过2的BST存在: False
瓶颈值不超过1的BST存在: False
瓶颈生成树的边: [(0, 1, 2), (0, 3, 1), (1, 2, 3), (1, 4, 5)]
线性时间算法找到的瓶颈生成树的边: [(0, 3, 1), (0, 1, 2), (1, 2, 3), (1, 4, 5)]
瓶颈值: 5


## 问题 3

**道路网（Road Network）**

假设有一个以图 $ G(V, E, l) $ 表示的道路网络，连接了一组城市 $ V $。我们假设该网络是有向的，并且每条道路 $(u, v) \in E$ 都有一个非负的长度 $ l(u, v) $。一条新的道路即将被建造，因此有一个列表 $ E' $ 包含它可以连接的城市对。每对 $(u, v) \in E'$ 都有一个对应的长度 $ l'(u, v) $。我们希望选择一对城市，使得两个城市 $ s, t \in V $ 之间的距离减少最大。请为此问题编写一个高效的算法，并详细解释算法的正确性和复杂度。


### 算法设计思路
1. 首先，需要计算原始图G中任意两点s和t之间的最短距离，可以使用Dijkstra算法从s和t分别作为源点计算到所有其他顶点的最短路径。
2. 对于每一条可能的新道路(u,v)∈E'，计算如果添加这条新道路后，s到t到最短路径长度会如何变化。
3. 新的最短路径可能是：
    - 原始的最短路径（不使用新道路）
    - 从s到u，然后通过新道路(u,v)，再从v到t
    - 从s到v，然后通过新道路(v,u)，再从u到t（如果E'中包含(v,u)）
4. 选择能使s到t之间距离减少最多的那条新道路。

### 算法实现

In [ ]:
def find_best_new_road(G, E_prime, s, t):
    dist_from_s = dijkstra(G, s)
    
    dist_from_t = dijkstra(G, t)
    
    original_dist = dist_from_s[t]
    
    max_reduction = 0
    best_road = None
    
    # 检查每条可能的新道路
    for u, v, length in E_prime:
        # 如果添加(u,v)，新的最短路径可能是s->u->v->t
        new_dist_with_uv = dist_from_s[u] + length + dist_from_t[v]
        
        reduction = original_dist - new_dist_with_uv
        
        if reduction > max_reduction and new_dist_with_uv < original_dist:
            max_reduction = reduction
            best_road = (u, v, length)
    
    return best_road, max_reduction

def dijkstra(G, start):
    dist = {v: float('infinity') for v in G.vertices}
    dist[start] = 0
    pq = [(0, start)]
    visited = set()
    
    while pq:
        current_dist, current_vertex = heapq.heappop(pq)
        
        if current_vertex in visited:
            continue
        
        visited.add(current_vertex)
        
        for neighbor, weight in G.get_neighbors(current_vertex):
            distance = current_dist + weight
            
            if distance < dist[neighbor]:
                dist[neighbor] = distance
                heapq.heappush(pq, (distance, neighbor))
    
    return dist

### 算法正确性
1. Dijikstra算法能够正确计算从一个顶点到所有其他顶点的最短路径（在边权重非负的情况下）。
2. 如果添加一条新道路(u,v)，那么从s到t到新最短路径要么是原来的路径，要么是经过这条新道路的路径。
3. 如果经过新道路，那么路径必然是`s→...→u→v→...→t`的形式，其长度是`dist_from_s[u] + length + dist_from_t[v]`。
4. 通过比较所有可能的新道路，我们可以减少距离最多的那条

### 时间复杂度
- 对s和t分别运行Dijkstra算法：O(|E| + |V|log|V|)
- 遍历所有可能的新道路：O(|E'|)，其中|E'|是可能的新道路数量
- 总时间复杂度：O(|E| + |V|log|V| + |E'|)

## 问题 4

**逃离问题**

一个 $ n \times n $ 的网格是一个无向图，由 $ n $ 行和 $ n $ 列的顶点组成，如下图所示。我们用 $(i,j)$ 表示第 $ i $ 行和第 $ j $ 列的顶点。除了边界顶点，网格中的所有顶点都有四个邻居，即满足 $ i = 1, i = n, j = 1 $ 或 $ j = n $ 的点 $(i,j)$。

给定网格中的 $ m \leq n^2 $ 个起点 $(x_1, y_1), (x_2, y_2), \cdots , (x_m, y_m)$，逃离问题是确定是否存在 $ m $ 条顶点不相交的路径（即路径之间不相交），从这些起点到边界上的任意 $ m $ 个不同点。例如，图1中的网格存在逃离。

(1) 该问题可以看作是一个最大流问题。考虑一个流网络，其中顶点和边都有容量。也就是说，进入任何给定顶点的总正流量受到容量限制。证明在具有边和顶点容量的网络中确定最大流可以简化为在具有可比大小的普通流网络上的最大流问题。更准确地说，你需要将一个具有顶点和边容量的网络 $ G = (V,E) $ 转换为另一个仅具有边容量的网络 $ G' = (V', E') $，使得两个网络上的最大流相同，并且你构建的新网络具有 $ V' = O(V) $ 个顶点和 $ E' = O(E) $ 条边。你可以假设网络是连通的。

(2) 描述一个解决逃离问题的高效算法，并分析其运行时间。


<div align="center"> <img alt="图片" src="./fig/escepe-p.png"> </div>
<center> 图2. 逃脱问题网格，起始顶点为黑色，其他网格顶点为白色</center>

idea：

### (1)证明：
将具有顶点和边容量的网络G=(V,E)转换为仅有边容量的网络G'=(V',E')：
对于G中的每个顶点v，在G'中创建两个顶点`v_in`和`v_out`：
- `v_in`接收所有进入v的流
- `v_out`发送所有从v出去的流
- 添加一条从`v_in`到`v_out`的边，容量等于v的顶点容量

**转换算法：**
```
函数 转换网络(G=(V,E))
    创建空图 G' = (V', E')
    
    对于每个 v ∈ V:
        添加顶点 v_in 到 V'
        添加顶点 v_out 到 V'
        添加边 (v_in, v_out) 到 E'，容量为 c(v)
    
    对于每条边 (u,v) ∈ E:
        添加边 (u_out, v_in) 到 E'，容量为 c(u,v)
    
    如果 s ∈ V 是源点:
        将 s_in 作为 G' 的源点
    如果 t ∈ V 是汇点:
        将 t_out 作为 G' 的汇点
    
    返回 G'
```
这样，G'中共有`|V'| = 2|V|`个顶点和`|E'| = |V| + |E|`条边，满足`|V'| = O(|V|)和|E'| = O(|E|)`。
在G'中，从源点s到汇点t的最大流与G中的最大流相同，因为：
- 任何在G中合法的流在G'中也是合法的
- G中的顶点容量限制在G'中通过`v_in`到`v_out`的边容量实现
= G中的边容量限制在G'中通过对应边的容量实现

### (2) 解决逃离问题
将逃离问题转化为最大流问题：
1. 构建流网络
2. 由于需要顶点不相交的路径，每个顶点的容量为1（除了源点和汇点）
3. 使用上面的转换将顶点容量转为边容量
4. 在转换后的网络上运行最大流算法
5. 如果最大流等于m，则存在m条顶点不相交的路径

**具体算法**
```
函数 解决逃离问题(n, m, 起点列表)
    // 构建网格图
    创建网格图G，大小为n×n
    添加超级源点s和超级汇点t
    
    // 连接起点和边界
    对于每个起点(x_k, y_k):
        添加边(s, (x_k, y_k))，容量为1
    
    对于每个边界点(i,j) (i=1或i=n或j=1或j=n):
        如果(i,j)不是起点:
            添加边((i,j), t)，容量为1
    
    // 连接相邻网格点
    对于每对相邻点(i,j)和(i',j'):
        添加边((i,j), (i',j'))，容量为1
        添加边((i',j'), (i,j))，容量为1
    
    // 转换顶点容量
    G' = 转换网络(G)  // 使用问题1的转换算法
    
    // 计算最大流
    最大流值 = 计算最大流(G', s, t)
    
    // 判断是否存在逃离
    返回 最大流值 == m
```

**时间复杂度**
- 构建网格图：O(n²)
- 转换顶点容量：O(n²)
- 计算最大流：使用Ford-Fulkerson算法，时间复杂度为O(|E|·f)，其中f是最大流值
- 在此问题中，|E| = O(n²)，f ≤ m ≤ n²
- 因此最大流计算的时间复杂度为O(n⁴)
总时间复杂度：O(n⁴)